# Set Up

In [1]:
# !pip install -qU transformers==4.48.3 datasets==2.18.0 optimum==1.24.0
# !pip install -qU json-repair==0.29.1

In [2]:
# !pip install transformers accelerate peft bitsandbytes

In [3]:
# !pip install -U bitsandbytes

In [4]:
# !pip install --force-reinstall "https://github.com/bitsandbytes-foundation/bitsandbytes/releases/download/continuous-release_multi-backend-refactor/bitsandbytes-0.44.1.dev0-py3-none-win_amd64.whl"

# Libraries Import

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
from os.path import join

In [7]:
from tqdm.auto import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [8]:
from data_model import getDetailExtractionMessage, getTranslationMessage
from functions import parse_json

In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
import torch

# Constants

In [10]:
BASE_MODEL_ID = "Qwen/Qwen2.5-1.5B-Instruct"
TORCH_TYPE = None

# Model (Base Model)

In [11]:
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_ID,
    device_map="auto",
    torch_dtype = TORCH_TYPE,
    offload_folder="./offload_dir"
)

tokenizer_temp = AutoTokenizer.from_pretrained(BASE_MODEL_ID)

In [12]:
base_model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((1536,), eps=1e-06)
    (rotary_emb): Qw

# Model Prediction

In [13]:
def predict(message: str):
    text = tokenizer_temp.apply_chat_template(
        message,
        tokenize=False,
        add_generation_prompt=True
    )

    model_inputs = tokenizer_temp([text], return_tensors="pt").to(base_model.device)

    generated_ids = base_model.generate(
        model_inputs.input_ids,
        max_new_tokens=1024,
        do_sample=False, top_k=None, temperature=None, top_p=None,
    )

    generated_ids = [
        output_ids[len(input_ids):]
        for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer_temp.batch_decode(generated_ids, skip_special_tokens=True)[0]

    return response

In [14]:
story = """
أنهى مسئولو النادى الأهلى كل تفاصيل عقد أحمد مصطفى زيزو، جناح الزمالك، بحيث ينضم اللاعب لمدة 4 سنوات قادمة بدءا من الموسم الجديد، فى صفقة انتقال حر بعد نهاية عقده مع الزمالك عقب نهاية الموسم الحالى.

ووضع الأهلى زيزو فى الفئة الأولى بحيث يحصل على 25 مليون جنيه سنويا بالإضافة إلى عقد إعلانى مقابل 60 مليون جنيه، بخلاف 80 مليون جنيه أخرى سيحصل عليها دفعة واحدة من خارج النادى عند الإعلان الرسمى عن الصفقة.

وينتهى عقد أحمد سيد زيزو مع الزمالك بنهاية الموسم الجارى، مما يجعل له الحق فى التفاوض مع أى نادٍ وإتمام الاتفاق معه والانضمام له عقب نهاية عقده الحالي خلال يونيو المقبل.

وحصل زيزو على تأشيرة السفر لأمريكا بالفعل للمشاركة مع الأهلى في مونديال الأندية الصيف المقبل.

ويستعد الأهلى للمشاركة فى بطولة كأس العالم للأندية بشكلها الجديد فى الولايات المتحدة الأمريكية، والمقامة بمشاركة 32 ناديًا من أكبر فرق العالم
"""

Before Tuning

In [15]:
getDetailExtractionMessage(story)

[{'role': 'system',
  'content': 'You are an NLP data paraser.\nYou will be provided by an Arabic text associated with a Pydantic scheme.\nGenerate the ouptut in the same story language.\nYou have to extract JSON details from text according the Pydantic details.\nExtract details as mentioned in text.\nDo not generate any introduction or conclusion.'},
 {'role': 'user',
  'content': '## Story:\nأنهى مسئولو النادى الأهلى كل تفاصيل عقد أحمد مصطفى زيزو، جناح الزمالك، بحيث ينضم اللاعب لمدة 4 سنوات قادمة بدءا من الموسم الجديد، فى صفقة انتقال حر بعد نهاية عقده مع الزمالك عقب نهاية الموسم الحالى.\n\nووضع الأهلى زيزو فى الفئة الأولى بحيث يحصل على 25 مليون جنيه سنويا بالإضافة إلى عقد إعلانى مقابل 60 مليون جنيه، بخلاف 80 مليون جنيه أخرى سيحصل عليها دفعة واحدة من خارج النادى عند الإعلان الرسمى عن الصفقة.\n\nوينتهى عقد أحمد سيد زيزو مع الزمالك بنهاية الموسم الجارى، مما يجعل له الحق فى التفاوض مع أى نادٍ وإتمام الاتفاق معه والانضمام له عقب نهاية عقده الحالي خلال يونيو المقبل.\n\nوحصل زيزو على تأشيرة

In [16]:
# predict(getDetailExtractionMessage(story))

In [17]:
getTranslationMessage(story)

[{'role': 'system',
  'content': 'You are a professional translator.\nYou will be provided by an Arabic text.\nYou have to translate the text into the `Targeted Language`.\nFollow the provided Scheme to generate a JSON\nDo not generate any introduction or conclusion.'},
 {'role': 'user',
  'content': '## Story:\nأنهى مسئولو النادى الأهلى كل تفاصيل عقد أحمد مصطفى زيزو، جناح الزمالك، بحيث ينضم اللاعب لمدة 4 سنوات قادمة بدءا من الموسم الجديد، فى صفقة انتقال حر بعد نهاية عقده مع الزمالك عقب نهاية الموسم الحالى.\n\nووضع الأهلى زيزو فى الفئة الأولى بحيث يحصل على 25 مليون جنيه سنويا بالإضافة إلى عقد إعلانى مقابل 60 مليون جنيه، بخلاف 80 مليون جنيه أخرى سيحصل عليها دفعة واحدة من خارج النادى عند الإعلان الرسمى عن الصفقة.\n\nوينتهى عقد أحمد سيد زيزو مع الزمالك بنهاية الموسم الجارى، مما يجعل له الحق فى التفاوض مع أى نادٍ وإتمام الاتفاق معه والانضمام له عقب نهاية عقده الحالي خلال يونيو المقبل.\n\nوحصل زيزو على تأشيرة السفر لأمريكا بالفعل للمشاركة مع الأهلى في مونديال الأندية الصيف المقبل.\n\nويستعد

In [18]:
# predict(getTranslationMessage(story))

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


'{\n  "translated_title": "Ahmed Moustafa Zizou\'s Contract Ends with Cairo Eagles",\n  "translated_content": "The management of Al Ahli has finalized all details regarding Ahmed Moustafa Zizou\'s contract, which he will join for four years starting from next season. The deal is free transfer after his contract expires with Al Ahli at the end of this season."\n}'

After Tuning

In [19]:
finetuned_model_id = "models"
base_model.load_adapter(finetuned_model_id)

ValueError: We need an `offload_dir` to dispatch this model according to this `device_map`, the following submodules need to be offloaded: model.layers.6, model.layers.7, model.layers.8, model.layers.9, model.layers.10, model.layers.11, model.layers.12, model.layers.13, model.layers.14, model.layers.15, model.layers.16, model.layers.17, model.layers.18, model.layers.19, model.layers.20, model.layers.21, model.layers.22, model.layers.23, model.layers.24, model.layers.25, model.layers.26, model.layers.27, model.norm, model.rotary_emb.